In [1]:
# Kütüphaneleri yüklüyoruz.

import cv2
import numpy as np

In [4]:
# Burada modeli yüklüyoruz. 
from ultralytics import YOLO
model = YOLO("yolov8l.pt")

In [5]:
font = cv2.FONT_HERSHEY_DUPLEX
kamera = cv2.VideoCapture("video1.mp4")



# Burada bölgeleri tanımlıyoruz. 
# İstenilen şekilde ve sayıda tanımlamabilir bu noktalar. Burada her bir nokta önceki ve sonraki nokta ile bağlantılı.
# Yani hem önceki hem de sonraki nokta ile kenar oluşturuyor bu noktalar

region1=np.array([(200,400),(0,600),(550,600),(600,400)])
region1 = region1.reshape((-1,1,2))


region2=np.array([(700,400),(750,600),(1250,600),(1200,400)])
region2 = region2.reshape((-1,1,2))





while True:
    
    # Her bir bölgede kaç tane araç olduğunu gösteren bir değişken tanımlıyoruz.
    # Her frame'de bunun değeri 0 olarak güncelleniyor.
    total1=0
    total2=0

    ret, frame = kamera.read()

    if not ret:
        break
        
        
    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model(rgb_img, verbose=False)
    labels=results[0].names
 

   
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        cls=results[0].boxes.cls[i]
       
    
        x1,y1,x2,y2,score,cls=int(x1),int(y1),int(x2),int(y2),float(score),int(cls)
        name=labels[cls]
        

        
        # Tespit edilen nesnenin score değeri %50'den küçükse ekranda gözükmeyecek
        if score<0.5:
            continue
        # Eğer nesne aşağıdaki araçlardan birisi değilse ekranda gözükmeyecek ve sayıma dahil değil
        if name!='car' and name!='truck'  and name!='motorcycle' and name!='bicycle' :
            continue
        
        # Nesnelerin orta noktasını buluyoruz.
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        
        # Nesnenin orta noktası 1. bölgenin içinde mi onu kontrol ediyoruz.
        # Eğer içindeyse bu değer 1 olacak
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        if inside_region1>0:
            # eğer nesne bu bölgeye girmişse o bölgeye ait değişkenin değeri 1 artıyor.
            total1+=1

        # Aynısını 2. bölge için yapıyoruzç
        inside_region2=cv2.pointPolygonTest(region2,(cx,cy),False)
        if inside_region2>0:
            total2+=1
        
        # Araçların orta noktasını daire ile gösteriyoruz.
        cv2.circle(frame,(cx,cy),10,(0,255,255),-1)
        
        # Araçları bir dikdörtgen ile gösteriyoruz.
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),5)
        
    # Her bir bölgede kaç tane araç olduğunu ekranda gösteriyoruz.
    # Buradaki koordonatlar bölgenin biraz üstü
    cv2.putText(frame, str(total1),(150, 380), font, 5, (0,0,255), 5)
    cv2.putText(frame, str(total2),(1150, 380), font, 5, (0,255,255), 5)
    
    # Bölgeleri ekranda gösteriyoruz
    cv2.polylines(frame,[region1],True,(255,0,0),8)
    cv2.polylines(frame,[region2],True,(255,0,255),8)

    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
kamera.release()
cv2.destroyAllWindows()